## SQL  StackOverflow

<img src="https://pictures.s3.yandex.net/resources/Frame_353_1_1664969703.png" alt="ER"/>

## 1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(*)
FROM stackoverflow.posts
WHERE (score > 300 OR  favorites_count >= 100) 
AND post_type_id = 1;

## 2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
SELECT ROUND(AVG(post_count), 0)
FROM (SELECT DISTINCT creation_date :: date,
              COUNT(*) OVER (PARTITION BY DATE_TRUNC('day', creation_date)) AS post_count
      FROM stackoverflow.posts
      WHERE post_type_id = 1
      AND creation_date :: date  BETWEEN '01.11.2008' AND '18.11.2008') AS prs;

## 3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
WITH sb AS (SELECT DISTINCT user_id,
                    CAST(MIN(creation_date) OVER (PARTITION BY user_id) AS date) AS point_date
            FROM stackoverflow.badges),
    su AS (SELECT id,
                  CAST(creation_date AS date)
          FROM stackoverflow.users)

SELECT COUNT(*)
FROM  sb
INNER JOIN su ON sb.user_id=su.id AND sb.point_date=su.creation_date;

## 4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT(post_id))
FROM stackoverflow.votes
WHERE post_id IN (SELECT id
FROM stackoverflow.posts
WHERE user_id IN (SELECT id
         FROM stackoverflow.users
         WHERE display_name LIKE '%Joel Coehoorn%'));

## 5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
        RANK() OVER(ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id;

## 6. Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
WITH sv AS (SELECT DISTINCT user_id,
      COUNT(*) OVER(PARTITION BY user_id) AS counter
      FROM stackoverflow.votes
      WHERE vote_type_id IN (SELECT id
                             FROM stackoverflow.vote_types
                             WHERE name = 'Close'))

SELECT *
FROM sv
ORDER BY counter DESC, user_id DESC
LIMIT 10;

## 7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.  

Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.  
  
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
WITH sv AS (SELECT user_id,
                   COUNT(id) AS badges
           FROM stackoverflow.badges
            WHERE CAST(creation_date AS date) BETWEEN '15.11.2008' AND '15.12.2008'
           GROUP BY user_id
           ORDER BY badges DESC
           LIMIT 10)
           
SELECT *,
        DENSE_RANK() OVER(ORDER BY badges DESC)
FROM sv
ORDER BY badges DESC, user_id;

## 8. Сколько в среднем очков получает пост каждого пользователя?  
Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.  
  
Не учитывайте посты без заголовка, а также те, что набрали ноль очков. 

In [ ]:
WITH sas AS (SELECT title,
     user_id,
     score,
     ROUND(AVG(score) OVER(PARTITION BY user_id), 0)
     FROM stackoverflow.posts
      WHERE title IS NOT NULL AND score <> 0       )
SELECT *
FROM sas;

## 9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM stackoverflow.posts
WHERE user_id IN (SELECT user_id
FROM stackoverflow.badges
GROUP BY user_id
HAVING COUNT(id) > 1000)
AND title IS NOT NULL;

## 10. Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.  
  
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
        views,
        CASE 
    WHEN views >= 350 THEN 1
    WHEN views >= 100 THEN 2
    ELSE 3
    END
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views <> 0;

## 11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH sus AS((SELECT id,
        views,
        CASE 
    WHEN views >= 350 THEN 1
    WHEN views >= 100 THEN 2
    ELSE 3
    END AS rang
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views <> 0
ORDER BY rang DESC, views, id)),

    sas AS (SELECT *,
            MAX(views) OVER(PARTITION BY rang) AS max_vi
            FROM sus)
            
SELECT id,
        rang,
        views
FROM sas
WHERE views = max_vi
ORDER BY views DESC, id;

## 12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
WITH sas AS (SELECT EXTRACT(DAY FROM CAST(creation_date AS timestamp)) AS day_num,
             COUNT(id) AS cont
             FROM stackoverflow.users
             WHERE CAST(DATE_TRUNC('month', creation_date) AS date) = '01.11.2008'
             GROUP BY day_num)

SELECT *,
        SUM(cont) OVER (ORDER BY day_num)
FROM sas
ORDER BY day_num;

## 13. Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
WITH sas AS (SELECT DISTINCT user_id,
             MIN(creation_date) OVER (PARTITION BY user_id)  AS first_post
             FROM stackoverflow.posts
            )



SELECT DISTINCT sas.user_id,
        first_post - creation_date
FROM stackoverflow.users AS sa
JOIN sas ON sas.user_id=sa.id;

## 14. Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT CAST(DATE_TRUNC('month', creation_date) AS date),
        SUM(views_count)
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date)= 2008
GROUP BY DATE_TRUNC('month', creation_date)
ORDER BY SUM(views_count) DESC;

## 15. Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT su.display_name,
        COUNT(DISTINCT sp.user_id)
FROM stackoverflow.posts AS sp
JOIN stackoverflow.users AS su ON su.id=sp.user_id
JOIN  stackoverflow.post_types AS pt ON pt.id=sp.post_type_id 
WHERE  sp.creation_date:: date BETWEEN su.creation_date:: date AND (su.creation_date:: date + INTERVAL '1 months')
AND pt.type LIKE '%Answer%'
GROUP BY su.display_name
HAVING COUNT(sp.id) > 100
ORDER BY su.display_name;


## 16. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH sas AS (SELECT DISTINCT sp.user_id
               FROM stackoverflow.posts AS sp
               JOIN stackoverflow.users AS su ON su.id=sp.user_id
               WHERE CAST(DATE_TRUNC('month', su.creation_date) AS date) = '01.09.2008'
               AND CAST(DATE_TRUNC('month', sp.creation_date) as date) = '01.12.2008')

SELECT CAST(DATE_TRUNC('month', creation_date) AS date) AS month_p,
        COUNT(id)
FROM stackoverflow.posts
WHERE user_id IN (SELECT *
    FROM sas)
GROUP BY month_p
ORDER BY month_p DESC;

## 17. Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.  
  
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id,
        creation_date,
        views_count,
        SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id, creation_date;

## 18. Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
SELECT ROUND(AVG(days_act))
FROM (SELECT  user_id,
COUNT(DISTINCT CAST(EXTRACT(DAY FROM CAST(creation_date AS timestamp)) AS numeric)) As days_act
FROM stackoverflow.posts
WHERE creation_date :: date BETWEEN '01.12.2008' AND '07.12.2008'
GROUP BY user_id) AS sus;

## 19. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- Номер месяца.
- Количество постов за месяц.
- Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.  
  
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
WITH sus AS (SELECT *,
             LAG(count_pres) OVER (ORDER BY n_mon) AS mon_past
    FROM (SELECT EXTRACT(MONTH FROM CAST(creation_date AS timestamp)) AS n_mon,
        COUNT(*) AS count_pres
FROM  stackoverflow.posts
WHERE creation_date:: date BETWEEN '01.09.2008' AND '31.12.2008'
GROUP BY n_mon) As bup)

SELECT n_mon,
        count_pres,
       ROUND((( count_pres:: numeric / mon_past) - 1) * 100, 2) AS procent
        
FROM sus;

## 20. 
Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
WITH top_user AS (SELECT user_id AS top,
                COUNT(*) AS posts
                  FROM stackoverflow.posts
                  GROUP BY user_id
                  ORDER BY posts DESC
                 LIMIT 1)

SELECT EXTRACT(WEEK FROM CAST(creation_date AS timestamp)) AS num_week,
        MAX(creation_date)
FROM stackoverflow.posts
WHERE user_id IN (SELECT top
                 FROM top_user)
AND CAST(DATE_TRUNC('month', creation_date) AS date) = '01.10.2008'
GROUP BY num_week;